# Definizione di Web Scraping






*Il web scraping è il processo di estrazione dei dati dai siti web. Può essere effettuato manualmente, ma più spesso viene automatizzato utilizzando strumenti software.*

In [ ]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 16.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get update
!apt-get install -y chromium-browser chromium-chromedriver

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,323 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,223 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-s

In [ ]:
%ls /usr/bin/

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [ ]:
def web_driver():
    options = webdriver.ChromeOptions()

    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920,1200")
    options.add_argument('--disable-dev-shm-usage')


    # Initialize the WebDriver instance with the specified options
    # ChromeDriver è a conti fatti il software installato che ci permetterà di pilotare il nostro browser
    # Driver fa uso di tale software ed è utilizzato per impartire degli ordini al browser
    driver = webdriver.Chrome(options=options)
    return driver

In [ ]:
def pulisci_stringa_con_vettore(stringa, parole_chiave):
    # Trova l'indice della prima parola chiave trovata nella stringa
    min_indice = len(stringa)
    for parola_chiave in parole_chiave:
        indice = stringa.find(parola_chiave)
        if indice != -1 and indice < min_indice:
            min_indice = indice
    # Se una parola chiave è trovata, ritorna la parte della stringa prima della parola chiave
    if min_indice != len(stringa):
        return stringa[:min_indice]
    # Se nessuna parola chiave è trovata, ritorna la stringa originale
    return stringa

#Scraping mediante SELENIUM dal sito Dica33

## Malattie cuore/sangue

In [ ]:
from datetime import datetime

def time_converter(data_testuale):
    """
    Converts a date string in Italian format to ISO 8601 format.

    Args:
        data_testuale (str): The date string in Italian format (e.g., "26 ottobre 2024").

    Returns:
        str: The date in ISO 8601 format (YYYY-MM-DD).
    """
    # Define a dictionary to map Italian month names to English month names
    italian_to_english_months = {
        "gennaio": "January",
        "febbraio": "February",
        "marzo": "March",
        "aprile": "April",
        "maggio": "May",
        "giugno": "June",
        "luglio": "July",
        "agosto": "August",
        "settembre": "September",
        "ottobre": "October",
        "novembre": "November",
        "dicembre": "December",
    }

    # Replace the Italian month name with the English month name
    for italian_month, english_month in italian_to_english_months.items():
        data_testuale = data_testuale.replace(italian_month, english_month)

    # Converti la data in un oggetto datetime
    data_datetime = datetime.strptime(data_testuale, "%d %B %Y")

    # Rimuovi l'ora e mantieni solo la data
    data_solo_data = data_datetime.date()

    # Converti la data in un formato ISO 8601 (senza ora)
    data_iso = data_solo_data.isoformat()
    return data_iso

# Esempio di utilizzo
data_testuale = "26 ottobre 2024"
data_iso = time_converter(data_testuale)
print(data_iso)
# Output: 2024-10-26


2024-10-26


In [ ]:
import pandas as pd

# 1. Initialization and Opening the Target URL:
driver = web_driver()
driver.get('https://www.dica33.it/esperto-risponde/domande-cuore-circolazione-e-malattie-del-sangue/')
parole_chiave = ["www.", "http://", "https://", "Dr." ,"Dott.", "Dott.ssa", "Cordiali", "Saluti", "Cordialmente", "Prego", "Arrivederci", "Prof."]
data = []
# 2. Loop to Navigate Through Pages:
for i in range(50):
    # 3. Extracting Question URLs:
    for urls in driver.find_elements(By.XPATH, '//*[@id="wrap"]/div/div/div/div[2]/div[1]/main/article/div[14]/div/div/h3/a'):
        url = urls.get_attribute("href")
        print("URL QA: {}".format(url))
        print()

        curr_driver = web_driver()
        curr_driver.get(url)

        # 4. Waiting for Elements to Load:
        WebDriverWait(curr_driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="wrap"]/div/div/div/div[2]/div[1]/main/article/div[15]'))
        )

        # 5. Extracting and Storing Data:
        try:
            date = curr_driver.find_element(By.XPATH, '//*[@id="wrap"]/div/div/div/div[2]/div[1]/main/article/div[15]/div[3]').text
            d = time_converter(date)
        except:
            d = None
            date = None

        try:
            question = curr_driver.find_element(By.XPATH, '//*[@id="wrap"]/div/div/div/div[2]/div[1]/main/article/div[15]').text.split('\n')[5]
        except:
            question = None

        try:
            answer = curr_driver.find_element(By.XPATH, '//*[@id="wrap"]/div/div/div/div[2]/div[1]/main/article/div[15]/div[4]/div').text
        except:
            answer = None

        print("Question Date: {}".format(d))
        print()
        print("Question: {}".format(question))
        print()
        print("Answer: {}".format(answer))
        print()

        data.append({
            "URL": url,
            "Question Date": d,
            "Question": question,
            "Answer": answer
        })

        # 6. Closing WebDriver Instances:
        curr_driver.quit()

    # 7. Navigating to the Next Page:
    next_page_script = "document.getElementById('pgg').value='{}'; paginazione.submit();".format(i + 2)
    driver.execute_script(next_page_script)

    print("Navigating to Next Page")
    print("-----------------------")
    print(i)

# 6. Closing WebDriver Instances:
driver.quit()

# Create DataFrame
df = pd.DataFrame(data)

NameError: name 'web_driver' is not defined

In [ ]:
df.head(10)

,URL,Question Date,Question,Answer Date,Answer,Doctor,Specialization,Location
0,https://www.miodottore.it/domande-risposte/don...,None,"Gentile utente, il Lexotan è specifico per i d...",None,"Gentile signora, stare in casa probabilmente l...",Pamela Busonero,"Psicologo, Psicoterapeuta",Firenze
1,https://www.miodottore.it/domande-risposte/ho-...,None,"Gentile signora, stare in casa probabilmente l...",None,Buongiorno. Dalle sue parole si evince il desi...,Silvia Pinna,"Psicologo, Psicoterapeuta, Tecnico sanitario",Roma
2,https://www.miodottore.it/domande-risposte/sal...,None,Buongiorno. Dalle sue parole si evince il desi...,None,Ciao. Da come descrivi quello che stai affront...,Nausicaa Precenzano,"Psicologo, Psicologo clinico, Psicoterapeuta",Bollate
3,https://www.miodottore.it/domande-risposte/sal...,None,Ciao. Da come descrivi quello che stai affront...,None,"Buongiorno, da quanto lei scrive mi sembra che...",Gaetana Bonasera,"Psicologo, Psicologo clinico, Psicoterapeuta",Roma
4,https://www.miodottore.it/domande-risposte/qua...,None,"Buongiorno, da quanto lei scrive mi sembra che...",None,"Buongiorno, sarebbe utile se lei descrivesse i...",Marco Lombardozzi,"Omeopata, Psicologo, Psicoterapeuta",Roma
5,https://www.miodottore.it/domande-risposte/e-d...,None,"Buongiorno, sarebbe utile se lei descrivesse i...",None,E' proprio un Disturbo da Attacchi di Panico c...,Massimo Carlo Mauri,"Psichiatra, Psicoterapeuta",Milano
6,https://www.miodottore.it/domande-risposte/buo...,None,E' proprio un Disturbo da Attacchi di Panico c...,None,Buonasera. Il Sequacor è SOLO UN FARMACO SINTO...,Massimo Milani,Cardiologo,Tortona
7,https://www.miodottore.it/domande-risposte/ho-...,None,Buonasera. Il Sequacor è SOLO UN FARMACO SINTO...,None,Salve quella che lei sta descrivendo sembra qu...,Filomena Coronella,Psicoterapeuta,San Marcellino
8,https://www.miodottore.it/domande-risposte/sal...,None,Salve quella che lei sta descrivendo sembra qu...,None,Ciao! È possibile che l'aumento di peso e il g...,Mauro Adragna,"Psichiatra, Psicoterapeuta",Palermo
9,https://www.miodottore.it/domande-risposte/buo...,None,Ciao! È possibile che l'aumento di peso e il g...,None,Buongiorno. Effettivamente è possibile che sia...,Ermanno Gioacchini,"Psichiatra, Psicoterapeuta",Roma


In [ ]:
# 1. Importing the re Module:
import re

new_data = []

# 2. Iterating over Rows of the DataFrame df:
for i, row in df.iterrows():
    raw_answer = row['Answer']

    # 3. Extracting Information using Regular Expressions:
    match_prof = re.search(r'Prof\.\s+((?:\w+\s?)+)', raw_answer)
    match_doctor = re.search(r'Dott\.\s+((?:\w+\s?)+)', raw_answer)

    # 4. Determining the Type of Professional and Name:
    signature = ""
    if match_doctor:
        doctor = match_doctor.group(1).strip()
        signature = "Dott"
    elif match_prof:
        doctor = match_prof.group(1).strip()
        signature = "Prof"
    else:
        doctor = None

    # 5. Extracting the Response Date:
    match_date = re.search(r'Risposta del (\d+ \w+ \d+)', raw_answer)


    # 6. Extracting Additional Information:
    match_job = re.search(r'{}\. \w+ \w+\n(.+)'.format(signature), raw_answer)
    match_specialization = re.search(r'Specialista in (.+)', raw_answer)
    match_location = re.search(r'(\w+ \(\w+\))$', raw_answer)

    # 7. Assigning Results of Regex Searches:
    if match_date:
        d = match_date.group(1).strip()
    else:
        date = None

    if match_job:
        job = match_job.group(1).strip()
    else:
        job = None

    if match_specialization:
        specialization = match_specialization.group(1).strip()
    else:
        specialization = None

    if match_location:
        location = match_location.group(1)
    else:
        location = None

    # 8. Extracting the Main Response:
    if location is not None:
        match_answer = re.search(r'Risposta a cura di:\n{}\..+?\n\n(.+?)\n{}\.'.format(signature, signature), raw_answer, re.DOTALL)
        if match_answer:
            answer = match_answer.group(1).replace('\n', ' ').strip()
        else:
            answer = None
    else:
        if signature == "Prof":
            start_index = match_prof.end()
            answer = raw_answer[start_index:].strip()
        elif signature == "Dott":
            start_index = match_doctor.end()
            answer = raw_answer[start_index:].strip()
        else:
            answer = None

    date_conv = time_converter(date)
    new_data.append({
            "URL": row["URL"],
            "Question Date": row["Question Date"],
            "Question": row["Question"],
            "Answer Date": date_conv,
            "Answer": answer,
            "Doctor": doctor,
            "Job": job,
            "Specialization": specialization,
            "Location": location
        })

# Create DataFrame
new_df = pd.DataFrame(new_data)

In [ ]:
new_df

,URL,Question Date,Question,Answer Date,Answer,Doctor,Job,Specialization,Location
0,https://www.dica33.it/esperto-risponde/domanda...,2024-10-26,Buonasera ho 28 anni in salute vado in palestr...,2017-05-07,"Le consiglio, dopo aver consultato il suo medi...",GIAN PIERO DI BARTOLOMEI,None,None,None
1,https://www.dica33.it/esperto-risponde/domanda...,2024-10-24,Salve vorrei chiedere informazioni riguardo al...,2017-05-07,Direi che il suo medico abbia ragione.,GIAN PIERO DI BARTOLOMEI,None,None,None
2,https://www.dica33.it/esperto-risponde/domanda...,2024-10-26,"Buongiorno, ho effettuato ecocardiogramma come...",2017-05-07,Il colesterolo con la mitriale non c’entra nie...,GIAN PIERO DI BARTOLOMEI,None,None,None
3,https://www.dica33.it/esperto-risponde/domanda...,2024-11-01,Buongiorno sono portatore di pacemaker e attua...,2017-05-07,Immagino che dovrà fare una scintigrafia cardi...,GIAN PIERO DI BARTOLOMEI,None,None,None
4,https://www.dica33.it/esperto-risponde/domanda...,2024-10-30,"Buonasera, sono un ragazzo di 27 anni(70kg per...",2017-05-07,Direi che il suo quadro lipidico è più che buo...,GIAN PIERO DI BARTOLOMEI,None,None,None
...,...,...,...,...,...,...,...,...,...
895,https://www.dica33.it/esperto-risponde/domanda...,2017-05-08,"Buongiorno, sono una donna di 35 anni, mamma, ...",2017-05-07,I dolori che descrive ( poco ) non fanno certo...,ALBERTO TITTOBELLO,Casa di cura privata,Gastroenterologia,Milano (MI)
896,https://www.dica33.it/esperto-risponde/domanda...,2017-05-08,Buongiorno dottori,2017-05-07,"Sia la sue pressione, sia la frequenza cardiac...",ALBERTO TITTOBELLO,Casa di cura privata,Gastroenterologia,Milano (MI)
897,https://www.dica33.it/esperto-risponde/domanda...,2017-05-07,trigliceridi - 311,2017-05-07,"Non c'è da preoccuparsi, ma dovrà iniziare sub...",ALBERTO TITTOBELLO,Casa di cura privata,Gastroenterologia,Milano (MI)
898,https://www.dica33.it/esperto-risponde/domanda...,2017-05-07,"Mia madre, 86 anni, dopo aver eseguito un TSA ...",2017-05-07,"Sì, le pare giusto. Le placche nelle arterie s...",ALBERTO TITTOBELLO,Casa di cura privata,Gastroenterologia,Milano (MI)


In [ ]:
new_df.to_csv("/content/drive/MyDrive/dica33_dataset_cuore.csv", index=False)

## Apparato respiratorio

In [ ]:
import pandas as pd

# 1. Initialization and Opening the Target URL:
driver = web_driver()
driver.get('https://www.dica33.it/esperto-risponde/domande-apparato-respiratorio/')
parole_chiave = ["www.", "http://", "https://", "Dr." ,"Dott.", "Dott.ssa", "Cordiali", "Saluti", "Cordialmente", "Prego", "Arrivederci", "Prof."]
data = []
# 2. Loop to Navigate Through Pages:
for i in range(50):
    # 3. Extracting Question URLs:
    for urls in driver.find_elements(By.XPATH, '//*[@id="wrap"]/div/div/div/div[2]/div[1]/main/article/div[14]/div/div/h3/a'):
        url = urls.get_attribute("href")
        print("URL QA: {}".format(url))
        print()

        curr_driver = web_driver()
        curr_driver.get(url)

        # 4. Waiting for Elements to Load:
        WebDriverWait(curr_driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="wrap"]/div/div/div/div[2]/div[1]/main/article/div[15]'))
        )

        # 5. Extracting and Storing Data:
        try:
            date = curr_driver.find_element(By.XPATH, '//*[@id="wrap"]/div/div/div/div[2]/div[1]/main/article/div[15]/div[3]').text
            d = time_converter(date)
        except:
            d = None
            date = None

        try:
            question = curr_driver.find_element(By.XPATH, '//*[@id="wrap"]/div/div/div/div[2]/div[1]/main/article/div[15]').text.split('\n')[5]
        except:
            question = None

        try:
            answer = curr_driver.find_element(By.XPATH, '//*[@id="wrap"]/div/div/div/div[2]/div[1]/main/article/div[15]/div[4]/div').text
        except:
            answer = None

        print("Question Date: {}".format(d))
        print()
        print("Question: {}".format(question))
        print()
        print("Answer: {}".format(answer))
        print()

        data.append({
            "URL": url,
            "Question Date": d,
            "Question": question,
            "Answer": answer
        })

        # 6. Closing WebDriver Instances:
        curr_driver.quit()

    # 7. Navigating to the Next Page:
    next_page_script = "document.getElementById('pgg').value='{}'; paginazione.submit();".format(i + 2)
    driver.execute_script(next_page_script)

    print("Navigating to Next Page")
    print("-----------------------")
    print()

# 6. Closing WebDriver Instances:
driver.quit()

# Create DataFrame
df = pd.DataFrame(data)

URL QA: https://www.dica33.it/esperto-risponde/domanda-rx-torace-316447.asp

Question Date: None

Question: Cosa significa questo referto:non lesioni pleuroparenchimali, sottili strie parenchimali sovradiaframmatica sinistra con seno costofrenico omolaterale obliterato in esiti di pleurite

Answer: Risposta del 19 dicembre 2023

Risposta a cura di:
Prof. MAURO GRANATA

Il referto si riferisce agli esiti di una pregressa infiammazione della pleura. Non sussistono altre problematiche degne di nota. Cordiali saluti.

URL QA: https://www.dica33.it/esperto-risponde/domanda-tosse-persistente-dopo-positivita-al-covid-311949.asp

Question Date: None

Question: Buon giorno, Dopo essere stato positivo al covid adesso sono negativo, ma ho una tosse secca e persistente Quali farmaci posso prendere?

Answer: Risposta del 07 novembre 2022

Risposta a cura di:
Prof. MAURO GRANATA

Prima di assumere qualsiasi terapia sarà opportuno parlare del problema col suo medico di medicina generale per la eventu

KeyboardInterrupt: 

In [ ]:
# 1. Importing the re Module:
import re

new_data = []

# 2. Iterating over Rows of the DataFrame df:
for i, row in df.iterrows():
    raw_answer = row['Answer']

    # 3. Extracting Information using Regular Expressions:
    match_prof = re.search(r'Prof\.\s+((?:\w+\s?)+)', raw_answer)
    match_doctor = re.search(r'Dott\.\s+((?:\w+\s?)+)', raw_answer)

    # 4. Determining the Type of Professional and Name:
    signature = ""
    if match_doctor:
        doctor = match_doctor.group(1).strip()
        signature = "Dott"
    elif match_prof:
        doctor = match_prof.group(1).strip()
        signature = "Prof"
    else:
        doctor = None

    # 5. Extracting the Response Date:
    match_date = re.search(r'Risposta del (\d+ \w+ \d+)', raw_answer)


    # 6. Extracting Additional Information:
    match_job = re.search(r'{}\. \w+ \w+\n(.+)'.format(signature), raw_answer)
    match_specialization = re.search(r'Specialista in (.+)', raw_answer)
    match_location = re.search(r'(\w+ \(\w+\))$', raw_answer)

    # 7. Assigning Results of Regex Searches:
    if match_date:
        d = match_date.group(1).strip()
    else:
        date = None

    if match_job:
        job = match_job.group(1).strip()
    else:
        job = None

    if match_specialization:
        specialization = match_specialization.group(1).strip()
    else:
        specialization = None

    if match_location:
        location = match_location.group(1)
    else:
        location = None

    # 8. Extracting the Main Response:
    if location is not None:
        match_answer = re.search(r'Risposta a cura di:\n{}\..+?\n\n(.+?)\n{}\.'.format(signature, signature), raw_answer, re.DOTALL)
        if match_answer:
            answer = match_answer.group(1).replace('\n', ' ').strip()
        else:
            answer = None
    else:
        if signature == "Prof":
            start_index = match_prof.end()
            answer = raw_answer[start_index:].strip()
        elif signature == "Dott":
            start_index = match_doctor.end()
            answer = raw_answer[start_index:].strip()
        else:
            answer = None

    date_conv = time_converter(date)
    new_data.append({
            "URL": row["URL"],
            "Question Date": row["Question Date"],
            "Question": row["Question"],
            "Answer Date": date_conv,
            "Answer": answer,
            "Doctor": doctor,
            "Job": job,
            "Specialization": specialization,
            "Location": location
        })

# Create DataFrame
new_df = pd.DataFrame(new_data)

In [ ]:
new_df.head(5)

,URL,Question Date,Question,Answer Date,Answer,Doctor,Job,Specialization,Location
0,https://www.dica33.it/esperto-risponde/domanda...,2023-12-18,Cosa significa questo referto:non lesioni pleu...,2020-12-05,Il referto si riferisce agli esiti di una preg...,MAURO GRANATA,None,None,None
1,https://www.dica33.it/esperto-risponde/domanda...,2022-11-02,"Buon giorno, Dopo essere stato positivo al cov...",2020-12-05,Prima di assumere qualsiasi terapia sarà oppor...,MAURO GRANATA,Medico Ospedaliero,Medicina interna,ROMA (RM)
2,https://www.dica33.it/esperto-risponde/domanda...,2022-10-12,L'addensamento polmonare è preoccupante? Cosa ...,2020-12-05,Dipende dalla natura della lesione. In genere ...,MAURO GRANATA,Medico Ospedaliero,Medicina interna,ROMA (RM)
3,https://www.dica33.it/esperto-risponde/domanda...,2022-10-05,Si può prendere Seractil 400 come antinfluenzale?,2020-12-05,Il Seractil è un antifiammatorio (FANS) non un...,FABIO TRECATE,Casa di cura convenzionata,Fisiatria,Milano (MI)
4,https://www.dica33.it/esperto-risponde/domanda...,2022-06-28,"Ciao! Ho 18 anni, soffro di ansia e stress da ...",2020-12-05,Gentile utente la dualità mente corpo rapprese...,MAURO GRANATA,Medico Ospedaliero,Medicina interna,ROMA (RM)


In [ ]:
new_df.to_csv("/content/drive/MyDrive/dica33_dataset_apparato_respiratorio.csv", index=False)

##Domande legate al Fegato

In [ ]:
import pandas as pd

# 1. Initialization and Opening the Target URL:
driver = web_driver()
driver.get('https://www.dica33.it/esperto-risponde/domande-fegato/')
parole_chiave = ["www.", "http://", "https://", "Dr." ,"Dott.", "Dott.ssa", "Cordiali", "Saluti", "Cordialmente", "Prego", "Arrivederci", "Prof."]
data = []
# 2. Loop to Navigate Through Pages:
for i in range(20):
    # 3. Extracting Question URLs:
    for urls in driver.find_elements(By.XPATH, '//*[@id="wrap"]/div/div/div/div[2]/div[1]/main/article/div[14]/div/div/h3/a'):
        url = urls.get_attribute("href")
        print("URL QA: {}".format(url))
        print()

        curr_driver = web_driver()
        curr_driver.get(url)

        # 4. Waiting for Elements to Load:
        WebDriverWait(curr_driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="wrap"]/div/div/div/div[2]/div[1]/main/article/div[15]'))
        )

        # 5. Extracting and Storing Data:
        try:
            date = curr_driver.find_element(By.XPATH, '//*[@id="wrap"]/div/div/div/div[2]/div[1]/main/article/div[15]/div[3]').text
            d = time_converter(date)
        except:
            d = None
            date = None

        try:
            question = curr_driver.find_element(By.XPATH, '//*[@id="wrap"]/div/div/div/div[2]/div[1]/main/article/div[15]').text.split('\n')[5]
        except:
            question = None

        try:
            answer = curr_driver.find_element(By.XPATH, '//*[@id="wrap"]/div/div/div/div[2]/div[1]/main/article/div[15]/div[4]/div').text
        except:
            answer = None

        print("Question Date: {}".format(d))
        print()
        print("Question: {}".format(question))
        print()
        print("Answer: {}".format(answer))
        print()

        data.append({
            "URL": url,
            "Question Date": d,
            "Question": question,
            "Answer": answer
        })

        # 6. Closing WebDriver Instances:
        curr_driver.quit()

    # 7. Navigating to the Next Page:
    next_page_script = "document.getElementById('pgg').value='{}'; paginazione.submit();".format(i + 2)
    driver.execute_script(next_page_script)

    print("Navigating to Next Page")
    print("-----------------------")
    print()

# 6. Closing WebDriver Instances:
driver.quit()

# Create DataFrame
df = pd.DataFrame(data)

Output streaming troncato alle ultime 5000 righe.
Dott. DOMENICO MADAMA

stia tranquilla. .

Dott. Domenico Madama

URL QA: https://www.dica33.it/esperto-risponde/domanda-hbsab-positivo-305529.asp

Question Date: 2021-10-13

Question: Salve, sono un ragazzo di 37 anni, ho eseguito degli esami del sangue è ho scoperto di essere positivo solamente all'hbsab, le transaminasi sono perfette, c'è pericolo? Grazie.

Answer: Risposta del 18 ottobre 2021

Risposta a cura di:
Dott. GIANFRANCO CROCE

Data la sua età è molto probabile che sia stato vaccinato per l'Epatite da HBV. La presenza dei soloi anticorpi HBsAb indica una immunità acquisita vaccinale. L'immunità acquisita dopo infezione oltre ad HBsAb mostra anche HBcAb. In mancanza di questi ultimi anticorpi si tratta dunque di Immunità Vaccinale e non di malattia né in atto né pregressa.
Perciò può stare tranquillo, non è malato. Consiglio di controllare la quantità di anticorpi HBsAb: se sono molto bassi potrebbe non essere perfettamente 

In [ ]:
# 1. Importing the re Module:
import re

new_data = []

# 2. Iterating over Rows of the DataFrame df:
for i, row in df.iterrows():
    raw_answer = row['Answer']

    # 3. Extracting Information using Regular Expressions:
    match_prof = re.search(r'Prof\.\s+((?:\w+\s?)+)', raw_answer)
    match_doctor = re.search(r'Dott\.\s+((?:\w+\s?)+)', raw_answer)

    # 4. Determining the Type of Professional and Name:
    signature = ""
    if match_doctor:
        doctor = match_doctor.group(1).strip()
        signature = "Dott"
    elif match_prof:
        doctor = match_prof.group(1).strip()
        signature = "Prof"
    else:
        doctor = None

    # 5. Extracting the Response Date:
    match_date = re.search(r'Risposta del (\d+ \w+ \d+)', raw_answer)


    # 6. Extracting Additional Information:
    match_job = re.search(r'{}\. \w+ \w+\n(.+)'.format(signature), raw_answer)
    match_specialization = re.search(r'Specialista in (.+)', raw_answer)
    match_location = re.search(r'(\w+ \(\w+\))$', raw_answer)

    # 7. Assigning Results of Regex Searches:
    if match_date:
        d = match_date.group(1).strip()
    else:
        date = None

    if match_job:
        job = match_job.group(1).strip()
    else:
        job = None

    if match_specialization:
        specialization = match_specialization.group(1).strip()
    else:
        specialization = None

    if match_location:
        location = match_location.group(1)
    else:
        location = None

    # 8. Extracting the Main Response:
    if location is not None:
        match_answer = re.search(r'Risposta a cura di:\n{}\..+?\n\n(.+?)\n{}\.'.format(signature, signature), raw_answer, re.DOTALL)
        if match_answer:
            answer = match_answer.group(1).replace('\n', ' ').strip()
        else:
            answer = None
    else:
        if signature == "Prof":
            start_index = match_prof.end()
            answer = raw_answer[start_index:].strip()
        elif signature == "Dott":
            start_index = match_doctor.end()
            answer = raw_answer[start_index:].strip()
        else:
            answer = None

    date_conv = time_converter(date)
    new_data.append({
            "URL": row["URL"],
            "Question Date": row["Question Date"],
            "Question": row["Question"],
            "Answer Date": date_conv,
            "Answer": answer,
            "Doctor": doctor,
            "Job": job,
            "Specialization": specialization,
            "Location": location
        })

# Create DataFrame
new_df = pd.DataFrame(new_data)

In [ ]:
new_df.head(5)

,URL,Question Date,Question,Answer Date,Answer,Doctor,Job,Specialization,Location
0,https://www.dica33.it/esperto-risponde/domanda...,2024-11-17,"Buongiorno, volevo chiederle una informazione....",2017-04-01,"Gentile signora, in linea generale, i valori a...",PIERGIORGIO BIONDANI,None,None,None
1,https://www.dica33.it/esperto-risponde/domanda...,2023-08-19,"Buongiorno, Sono stata operata il 18 Luglio di...",2017-04-01,Ovviamente esiste una serie di esami di labora...,MARIO GUSLANDI,Gastroenterologia,Gastroenterologia,MILANO (MI)
2,https://www.dica33.it/esperto-risponde/domanda...,2023-08-03,"Salve, ho fatto un ecografia all'addome ed il ...",2017-04-01,Urge Tac con mezzo di contrasto,MARIO GUSLANDI,Gastroenterologia,Gastroenterologia,MILANO (MI)
3,https://www.dica33.it/esperto-risponde/domanda...,2023-07-10,"Buongiorno, Sono impanicato. Il medico curante...",2017-04-01,Prima di tutto deve eseguire una ecografia add...,MARIO GUSLANDI,Gastroenterologia,Gastroenterologia,MILANO (MI)
4,https://www.dica33.it/esperto-risponde/domanda...,2023-06-12,"Buongiorno, sono ipertiroidea l endocrinologa ...",2017-04-01,La gamma-gt alta non dipende dal tapazole e ri...,MARIO GUSLANDI,Gastroenterologia,Gastroenterologia,MILANO (MI)


In [ ]:
new_df.to_csv("/content/drive/MyDrive/dica33_dataset_fegato.csv", index=False)

## Alimentazione / Naso&MalDiGola / Mente&Cervello

In [ ]:
import pandas as pd

#https://www.dica33.it/esperto-risponde/domande-alimentazione//
#https://www.dica33.it/esperto-risponde/domande-orecchie-naso-e-gola//
#https://www.dica33.it/esperto-risponde/domande-mente-e-cervello

# 1. Initialization and Opening the Target URL:
driver = web_driver()
driver.get('https://www.dica33.it/esperto-risponde/domande-mente-e-cervello')
parole_chiave = ["www.", "http://", "https://", "Dr." ,"Dott.", "Dott.ssa", "Cordiali", "Saluti", "Cordialmente", "Prego", "Arrivederci", "Prof."]
data = []
# 2. Loop to Navigate Through Pages:
for i in range(100):
    # 3. Extracting Question URLs:
    for urls in driver.find_elements(By.XPATH, '//*[@id="wrap"]/div/div/div/div[2]/div[1]/main/article/div[14]/div/div/h3/a'):
        url = urls.get_attribute("href")
        print("URL QA: {}".format(url))
        print()

        curr_driver = web_driver()
        curr_driver.get(url)

        # 4. Waiting for Elements to Load:
        WebDriverWait(curr_driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="wrap"]/div/div/div/div[2]/div[1]/main/article/div[15]'))
        )

        # 5. Extracting and Storing Data:
        try:
            date = curr_driver.find_element(By.XPATH, '//*[@id="wrap"]/div/div/div/div[2]/div[1]/main/article/div[15]/div[3]').text
            d = time_converter(date)
        except:
            d = None
            date = None

        try:
            question = curr_driver.find_element(By.XPATH, '//*[@id="wrap"]/div/div/div/div[2]/div[1]/main/article/div[15]').text.split('\n')[5]
        except:
            question = None

        try:
            answer = curr_driver.find_element(By.XPATH, '//*[@id="wrap"]/div/div/div/div[2]/div[1]/main/article/div[15]/div[4]/div').text
        except:
            answer = None

        print("Question Date: {}".format(d))
        print()
        print("Question: {}".format(question))
        print()
        print("Answer: {}".format(answer))
        print()

        data.append({
            "URL": url,
            "Question Date": d,
            "Question": question,
            "Answer": answer
        })

        # 6. Closing WebDriver Instances:
        curr_driver.quit()

    # 7. Navigating to the Next Page:
    next_page_script = "document.getElementById('pgg').value='{}'; paginazione.submit();".format(i + 2)
    driver.execute_script(next_page_script)

    print("Navigating to Next Page")
    print("-----------------------")
    print(i)

# 6. Closing WebDriver Instances:
driver.quit()

# Create DataFrame
df = pd.DataFrame(data)

Output streaming troncato alle ultime 5000 righe.
La prima ipotesi che verrebbe da fare è che in quei momenti si attivi una iperventilazione e che questa produca l'alterazione muscolare dovuta al cambiamento del rapporto tra ossigeno ed anidrite carbonica nel sangue. Può provare a fare respiri profondi all'interno di un sacchetto di carta, in genere questo migliora le cose molto rapidamente e senza effetti collaterali, in alternativa anche piccoli dosaggi di benzodiazepine (diazepam, lorazepam, alprazolam) ma il respirare con naso e bocca all'interno di un sacchetto di carta è più efficace.

Dott. Angelo Grossi
Casa di cura privata
Specialista attività privata
Specialista in Psichiatria
Bologna (BO)

URL QA: https://www.dica33.it/esperto-risponde/domanda-pensieri-ossessivi-219009.asp

Question Date: 2013-01-27

Question: Buongiorno, volevo sapere, sono in cura con tavor da un mese per ansia adesso va meglio ma quando mi alzo la mattina ho sempre pensieri nella testa come se stessi asco

In [ ]:
# 1. Importing the re Module:
import re

new_data = []

# 2. Iterating over Rows of the DataFrame df:
for i, row in df.iterrows():
    raw_answer = row['Answer']

    # 3. Extracting Information using Regular Expressions:
    match_prof = re.search(r'Prof\.\s+((?:\w+\s?)+)', raw_answer)
    match_doctor = re.search(r'Dott\.\s+((?:\w+\s?)+)', raw_answer)

    # 4. Determining the Type of Professional and Name:
    signature = ""
    if match_doctor:
        doctor = match_doctor.group(1).strip()
        signature = "Dott"
    elif match_prof:
        doctor = match_prof.group(1).strip()
        signature = "Prof"
    else:
        doctor = None

    # 5. Extracting the Response Date:
    match_date = re.search(r'Risposta del (\d+ \w+ \d+)', raw_answer)


    # 6. Extracting Additional Information:
    match_job = re.search(r'{}\. \w+ \w+\n(.+)'.format(signature), raw_answer)
    match_specialization = re.search(r'Specialista in (.+)', raw_answer)
    match_location = re.search(r'(\w+ \(\w+\))$', raw_answer)

    # 7. Assigning Results of Regex Searches:
    if match_date:
        d = match_date.group(1).strip()
    else:
        date = None

    if match_job:
        job = match_job.group(1).strip()
    else:
        job = None

    if match_specialization:
        specialization = match_specialization.group(1).strip()
    else:
        specialization = None

    if match_location:
        location = match_location.group(1)
    else:
        location = None

    # 8. Extracting the Main Response:
    if location is not None:
        match_answer = re.search(r'Risposta a cura di:\n{}\..+?\n\n(.+?)\n{}\.'.format(signature, signature), raw_answer, re.DOTALL)
        if match_answer:
            answer = match_answer.group(1).replace('\n', ' ').strip()
        else:
            answer = None
    else:
        if signature == "Prof":
            start_index = match_prof.end()
            answer = raw_answer[start_index:].strip()
        elif signature == "Dott":
            start_index = match_doctor.end()
            answer = raw_answer[start_index:].strip()
        else:
            answer = None

    date_conv = time_converter(date)
    new_data.append({
            "URL": row["URL"],
            "Question Date": row["Question Date"],
            "Question": row["Question"],
            "Answer Date": date_conv,
            "Answer": answer,
            "Doctor": doctor,
            "Job": job,
            "Specialization": specialization,
            "Location": location
        })

# Create DataFrame
new_df = pd.DataFrame(new_data)

In [ ]:
new_df

,URL,Question Date,Question,Answer Date,Answer,Doctor,Job,Specialization,Location
0,https://www.dica33.it/esperto-risponde/domanda...,2024-10-18,Buongiorno. Mi sono sottoposta a una risonanza...,2011-10-21,None,None,None,None,None
1,https://www.dica33.it/esperto-risponde/domanda...,2024-10-20,"Buon giorno, a febbraio 2024 ho fatto una angi...",2011-10-21,None,None,None,None,None
2,https://www.dica33.it/esperto-risponde/domanda...,2024-10-23,"Gentili dottori, a seguito di debolezza alle g...",2011-10-21,None,None,None,None,None
3,https://www.dica33.it/esperto-risponde/domanda...,2024-08-22,"Buonasera, assumo benzodiazepine da più di 45 ...",2011-10-21,"Gentile utente, la ringrazio per la sua domand...",FEDERICO BARANZINI,None,None,None
4,https://www.dica33.it/esperto-risponde/domanda...,2024-08-23,Buongiorno Dottori attualmente sono in cura pr...,2011-10-21,"Buongiorno, come medico psichiatra, comprendo ...",FEDERICO BARANZINI,None,None,None
...,...,...,...,...,...,...,...,...,...
1855,https://www.dica33.it/esperto-risponde/domanda...,2011-10-28,Il ZINCOVAL mi è stato detto che aiuta negli s...,2011-10-21,Il prodotto da lei citato è omeopatico per cui...,Ssa Maria Letizia Primo\nSpecialista attività ...,None,Medicina legale,Torino (TO)
1856,https://www.dica33.it/esperto-risponde/domanda...,2011-10-27,salve. Ho 25anni. e da tempo sento come se fos...,2011-10-21,"Le dica quello che ha scritto qui, non la trov...",MARCO PAOLEMILI,Casa di cura convenzionata,Psichiatria,Roma (RM)
1857,https://www.dica33.it/esperto-risponde/domanda...,2011-10-27,"salve,",2011-10-21,gentile signora per prima cosa è necessario ch...,Ssa Maria Letizia Primo\nSpecialista attività ...,None,Medicina legale,Torino (TO)
1858,https://www.dica33.it/esperto-risponde/domanda...,2011-10-20,"Salve Dottore,",2011-10-21,gentile signora le consiglierei una visita odo...,Ssa Maria Letizia Primo\nSpecialista attività ...,None,Medicina legale,Torino (TO)


In [ ]:
#new_df.to_csv("/content/drive/MyDrive/dica33_dataset_alimentazione.csv", index=False)
#new_df.to_csv("/content/drive/MyDrive/dica33_dataset_naso.csv", index=False)
new_df.to_csv("/content/drive/MyDrive/dica33_dataset_mente.csv", index=False)

#Scraping sito MioDottore

## Domande legate all'ansia

Nota sull'xpath: /html/body/div[3]/div[2]/main/div[2]/section[3]/div[2]/div[1]/div[1]/div

L'XPath che ottieni dall'esploratore risorse del browser è un XPath assoluto, che rappresenta il percorso completo dall'elemento radice (<html>) fino all'elemento desiderato. Questo tipo di XPath è molto specifico e può diventare fragile se la struttura della pagina cambia.
Per rendere il tuo XPath più robusto, è consigliabile utilizzare un XPath relativo che si basa su attributi unici degli elementi. Ad esempio, se l'elemento che stai cercando ha un attributo class o id unico, puoi usarlo per creare un XPath più affidabile:

//div[@class='h4 font-weight-normal doctor-question-body mb-2']/a[@class='text-body']

Nota sul ''main''
In HTML, <main> è un elemento semantico che rappresenta il contenuto principale del documento. Questo elemento è utilizzato per racchiudere il contenuto principale della pagina, escludendo elementi come intestazioni, barre laterali, piè di pagina e altri contenuti ripetitivi. L'uso di <main> aiuta i motori di ricerca e gli screen reader a comprendere meglio la struttura della pagina e a identificare rapidamente il contenuto principale.

In [ ]:
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 1. Inizializzazione e Apertura dell'URL di Destinazione:
driver = web_driver()
# 1.1 Accedo all'indirizzo web da cui voglio estrarre le informazioni
driver.get('https://www.miodottore.it/patologie/ansia/domande')
parole_chiave = ["www.", "http://", "https://", "Dr." ,"Dott.", "Dott.ssa", "Cordiali", "Saluti", "Cordialmente", "Prego", "Arrivederci", "Prof."]
data = []
# 2. Loop per navigare tra le pagine
for i in range(20):
    # 3. Estrazione URL per le domande specificando l'xpath di riferimento sfruttando la libreria By



    for urls in driver.find_elements(By.XPATH, "//div[contains(@class, 'doctor-question-body')]/a[contains(@class, 'text-body')]"):
        #3.1. Estraggo l'attributo per navigare tra le pagine che corrisponde a href
        url = urls.get_attribute("href")
        print("URL QA: {}".format(url))
        print()

        curr_driver = web_driver()
        curr_driver.get(url)

        # 4. Apertura URL domande e estrazione dati:
        WebDriverWait(curr_driver, 20).until(
            EC.presence_of_element_located((By.XPATH, "//main"))
        )


        #5. Extracting and Storing Data:
        #5.1 Salvo la domanda
        try:
            question = curr_driver.find_element(By.XPATH, "//div[@class='h4 font-weight-normal doctor-question-body mb-2']").text
            question_senza_acapo = question.replace('\n', ' ')
            # Rimuovi eventuali spazi extra testo_senza_acapo = ' '
            question_senza_acapo = ' '.join(answer_senza_acapo.split())
        except:
            question = None
            question_senza_acapo = None
        #5.3 Salvo la risposta
        try:
            answer = curr_driver.find_element(By.XPATH, "//div[contains(@class, 'doctor-answer-content') and contains(@class, 'mb-1')]/div" ).text
            stringa_pulita = pulisci_stringa_con_vettore(answer, parole_chiave)
            # Sostituisci gli a capo con uno spazio
            answer_senza_acapo = stringa_pulita.replace('\n', ' ')
            # Rimuovi eventuali spazi extra testo_senza_acapo = ' '
            answer_senza_acapo = ' '.join(answer_senza_acapo.split())
        except:
            answer = None
            answer_senza_acapo = None

        try:
            dottore = curr_driver.find_element(By.XPATH, '//span[@itemprop="name"]').text
        except:
            dottore = None

        try:
            specializzazione = curr_driver.find_element(By.XPATH, "//div[contains(@class, 'small text-muted')]" ).text
        except:
            specializzazione = None

        try:
            Location = curr_driver.find_element(By.XPATH, "//div[contains(@class, 'small text-placeholder')]" ).text
        except:
            Location = None

        question_date = None
        answer_date = None
        # Rimuovi i titoli "Dott.ssa", "Dott." e "Dr." dal nome
        dottore = dottore.replace('Dott.ssa ', '').replace('Dott. ', '').replace('Dott. ', '').replace('Dr. ', '')

        print("Question: {}".format(question_senza_acapo))
        print("Answer: {}".format(answer_senza_acapo))
        print("Doctor: {}".format(dottore))
        print("Specialization: {}".format(specializzazione))
        print("Location: {}".format(Location))
        print()

        data.append({
            "URL": url,
            "Question Date" : question_date,
            "Question": question_senza_acapo,
            "Answer Date": answer_date,
            "Answer": answer_senza_acapo,
            "Doctor": dottore,
            "Specialization": specializzazione,
            "Location": Location
        })

        # 6. Closing WebDriver Instances:
        curr_driver.quit()


    # Trova l'elemento "Avanti" e cliccaci sopra
    button = driver.find_element(By.XPATH, "//li[contains(@class, 'page-item')]/a[contains(@class, 'page-link') and contains(@class, 'page-link-next') and contains(@class, 'd-flex') and contains(@class, 'align-items-center') and contains(@class, 'justify-content-center')]")
    driver.execute_script("arguments[0].click();", button)


    print("Navigating to Next Page")
    print("-----------------------")
    print("-----------------------")
    print()

# 6. Closing WebDriver Instances:
driver.quit()

# Salvo il tutto su df=dataframe, struttura dati tipica di pandas
df = pd.DataFrame(data)

URL QA: https://www.miodottore.it/domande-risposte/donna-54-anni-in-depressione-e-poi-ansia-da-un-anno-e-mezzo-in-cura-con-antidepressivi-e-ansiolitici

Question: Gentile utente, il Lexotan è specifico per i disturbi d'ansia così come il Remoren lo è per il trattamento della depressione. Non necessariamente potrebbe riscontrare gli stessi effetti collaterali di sua madre poiché gli stessi variano da persona a persona e sono molto soggettivi. Mi soffermo però sulla parte in cui scrive "stato di pre-depressione a mio avviso" che cosa intende? Quali sintomi sta vivendo per ritenere necessaria la prescrizione del farmaco? Ha avuto la possibilità di effettuare un consulto psicologico ora o in passato?
Answer: Gentile signora, stare in casa probabilmente la mette di fronte a se stessa, ed in un momento della sua vita cosi delicato questo le crea ansia e panico. Sta facendo un percorso con un terapeuta, questo è molto importante perché le da la spinta per uscire da questo brutto periodo. Prob

In [ ]:
df.head(5)

,URL,Question Date,Question,Answer Date,Answer,Doctor,Specialization,Location
0,https://www.miodottore.it/domande-risposte/don...,None,"Gentile utente, il Lexotan è specifico per i d...",None,"Gentile signora, stare in casa probabilmente l...",Pamela Busonero,"Psicologo, Psicoterapeuta",Firenze
1,https://www.miodottore.it/domande-risposte/ho-...,None,"Gentile signora, stare in casa probabilmente l...",None,Buongiorno. Dalle sue parole si evince il desi...,Silvia Pinna,"Psicologo, Psicoterapeuta, Tecnico sanitario",Roma
2,https://www.miodottore.it/domande-risposte/sal...,None,Buongiorno. Dalle sue parole si evince il desi...,None,Ciao. Da come descrivi quello che stai affront...,Nausicaa Precenzano,"Psicologo, Psicologo clinico, Psicoterapeuta",Bollate
3,https://www.miodottore.it/domande-risposte/sal...,None,Ciao. Da come descrivi quello che stai affront...,None,"Buongiorno, da quanto lei scrive mi sembra che...",Gaetana Bonasera,"Psicologo, Psicologo clinico, Psicoterapeuta",Roma
4,https://www.miodottore.it/domande-risposte/qua...,None,"Buongiorno, da quanto lei scrive mi sembra che...",None,"Buongiorno, sarebbe utile se lei descrivesse i...",Marco Lombardozzi,"Omeopata, Psicologo, Psicoterapeuta",Roma


In [ ]:
df.to_csv("/content/drive/MyDrive/miodottore_dataset_ansia.csv", index=False)

##Domande sulla Depressione

In [ ]:
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 1. Inizializzazione e Apertura dell'URL di Destinazione:
driver = web_driver()
# 1.1 Accedo all'indirizzo web da cui voglio estrarre le informazioni
driver.get('https://www.miodottore.it/patologie/depressione/domande')
parole_chiave = ["www.", "http://", "https://", "Dr." ,"Dott.", "Dott.ssa", "Cordiali", "Saluti", "Cordialmente", "Prego", "Arrivederci", "Prof."]
data = []
# 2. Loop per navigare tra le pagine
for i in range(30):
    # 3. Estrazione URL per le domande specificando l'xpath di riferimento sfruttando la libreria By



    for urls in driver.find_elements(By.XPATH, "//div[contains(@class, 'doctor-question-body')]/a[contains(@class, 'text-body')]"):
        #3.1. Estraggo l'attributo per navigare tra le pagine che corrisponde a href
        url = urls.get_attribute("href")
        print("URL QA: {}".format(url))
        print()

        curr_driver = web_driver()
        curr_driver.get(url)

        # 4. Apertura URL domande e estrazione dati:
        WebDriverWait(curr_driver, 20).until(
            EC.presence_of_element_located((By.XPATH, "//main"))
        )


        #5. Extracting and Storing Data:
        #5.1 Salvo la domanda
        try:
            question = curr_driver.find_element(By.XPATH, "//div[@class='h4 font-weight-normal doctor-question-body mb-2']").text
            # Sostituisci gli a capo con uno spazio
            question_senza_acapo = question.replace('\n', ' ')
            # Rimuovi eventuali spazi extra testo_senza_acapo = ' '
            question_senza_acapo = ' '.join(answer_senza_acapo.split())
        except:
            question = None
            question_senza_acapo = None
        #5.3 Salvo la risposta
        try:
            answer = curr_driver.find_element(By.XPATH, "//div[contains(@class, 'doctor-answer-content') and contains(@class, 'mb-1')]/div" ).text
            stringa_pulita = pulisci_stringa_con_vettore(answer, parole_chiave)
            # Sostituisci gli a capo con uno spazio
            answer_senza_acapo = stringa_pulita.replace('\n', ' ')
            # Rimuovi eventuali spazi extra testo_senza_acapo = ' '
            answer_senza_acapo = ' '.join(answer_senza_acapo.split())
        except:
            answer = None
            answer_senza_acapo = None

        try:
            dottore = curr_driver.find_element(By.XPATH, '//span[@itemprop="name"]').text
        except:
            dottore = None

        try:
            specializzazione = curr_driver.find_element(By.XPATH, "//div[contains(@class, 'small text-muted')]" ).text
        except:
            specializzazione = None

        try:
            Location = curr_driver.find_element(By.XPATH, "//div[contains(@class, 'small text-placeholder')]" ).text
        except:
            Location = None

        question_date = None
        answer_date = None
        # Rimuovi i titoli "Dott.ssa", "Dott." e "Dr." dal nome
        dottore = dottore.replace('Dott.ssa ', '').replace('Dott. ', '').replace('Dott. ', '').replace('Dr. ', '')

        print("Question: {}".format(question_senza_acapo))
        print("Answer: {}".format(answer_senza_acapo))
        print("Doctor: {}".format(dottore))
        print("Specialization: {}".format(specializzazione))
        print("Location: {}".format(Location))
        print()

        data.append({
            "URL": url,
            "Question Date" : question_date,
            "Question": question_senza_acapo,
            "Answer Date": answer_date,
            "Answer": answer_senza_acapo,
            "Doctor": dottore,
            "Specialization": specializzazione,
            "Location": Location
        })

        # 6. Closing WebDriver Instances:
        curr_driver.quit()


    # Trova l'elemento "Avanti" e cliccaci sopra
    button = driver.find_element(By.XPATH, "//li[contains(@class, 'page-item')]/a[contains(@class, 'page-link') and contains(@class, 'page-link-next') and contains(@class, 'd-flex') and contains(@class, 'align-items-center') and contains(@class, 'justify-content-center')]")
    driver.execute_script("arguments[0].click();", button)


    print("Navigating to Next Page")
    print("-----------------------")
    print("-----------------------")
    print()

# 6. Closing WebDriver Instances:
driver.quit()

# Salvo il tutto su df=dataframe, struttura dati tipica di pandas
df = pd.DataFrame(data)

URL QA: https://www.miodottore.it/domande-risposte/ho-62-anni-un-compagno-che-mi-vuole-bene-e-con-il-quale-sto-bene-una-casa-mia-e-niente-di-cui-possa

Question: Gentile Utente, Le confermo che la RM encefalica è perfettamente normale (le linee guida del sito non permettono di inviare immagini). Questa normalità non mi sorprende in quanto nella stragrande maggioranza delle cefalee la RM encefalica è negativa, cioè normale. Delle visite che ha fatto mi pare che manchi quella neurologica ("Sono stato da tutti: Oculista, otorino, ortopedico, internista") che poi è quella più indicata per le cefalee, per cui La invito a rivolgersi ad un collega esperto proprio in cefalee.
Answer: Gentile signora, Si coglie che sta vivendo una situazione di difficoltà, tuttavia può essere opportuno attendere ancora un po' di tempo prima che le terapie che ha messo in moto abbiano un effetto. Un percorso psicologico può richiedere anche un tempo lungo e gli effetti positivi non arrivano sempre immediatamente

In [ ]:
df.head(5)

,URL,Question Date,Question,Answer Date,Answer,Doctor,Specialization,Location
0,https://www.miodottore.it/domande-risposte/ho-...,None,"Gentile Utente, Le confermo che la RM encefali...",None,"Gentile signora, Si coglie che sta vivendo una...",Alessia Vaudano,"Psicologo, Psicoterapeuta, Psicologo clinico",Torino
1,https://www.miodottore.it/domande-risposte/buo...,None,"Gentile signora, Si coglie che sta vivendo una...",None,In genere la duloxetina è un farmaco abbastanz...,Giuseppe Gervasio,"Psichiatra, Psicoterapeuta",Frattamaggiore
2,https://www.miodottore.it/domande-risposte/buo...,None,In genere la duloxetina è un farmaco abbastanz...,None,"Buongiorno, l'aumento di peso è un effetto ind...",Fabio Misiano,"Psichiatra, Psicoterapeuta",Roma
3,https://www.miodottore.it/domande-risposte/io-...,None,"Buongiorno, l'aumento di peso è un effetto ind...",None,"Buongiorno, se non ha fatto un recente esame d...",Ermanno Gioacchini,"Psichiatra, Psicoterapeuta",Roma
4,https://www.miodottore.it/domande-risposte/sal...,None,"Buongiorno, se non ha fatto un recente esame d...",None,"Gentile utente, Ezequa, che contiene il princi...",Simone Fossella,"Psichiatra, Psicoterapeuta",Roma


In [ ]:
df.to_csv("/content/drive/MyDrive/miodottore_dataset_depressione.csv", index=False)

#Scraping sito MEDICITALIA

*A causa di alcuni problemi col reperimento dei dati è stato modificato il codice rispetto ai casi di scraping precedenti, si è sfruttato un driver singolo, poiché il doppio driver dava problemi*



## Domande sulla Cefalea / Colon Irritabile / Alimentazione

In [ ]:
def pulisci_stringa_con_vettore(stringa, parole_chiave):
    # Trova l'indice della prima parola chiave trovata nella stringa
    min_indice = len(stringa)
    for parola_chiave in parole_chiave:
        indice = stringa.find(parola_chiave)
        if indice != -1 and indice < min_indice:
            min_indice = indice
    # Se una parola chiave è trovata, ritorna la parte della stringa prima della parola chiave
    if min_indice != len(stringa):
        return stringa[:min_indice]
    # Se nessuna parola chiave è trovata, ritorna la stringa originale
    return stringa






In [ ]:
import re

def rimuovi_data_ora(testo):
    # Definisci il pattern regex per la data e l'ora
    pattern = r'\d{2}\.\d{2}\.\d{4} \d{2}:\d{2}'

    # Sostituisci il pattern con una stringa vuota
    testo_pulito = re.sub(pattern, '', testo)

    # Rimuovi eventuali spazi extra
    testo_pulito = testo_pulito.strip()

    return testo_pulito

# Esempio di utilizzo
testo_originale = "Questo è un esempio di testo con una data 25.01.2019 08:50 inclusa."
testo_pulito = rimuovi_data_ora(testo_originale)
print(testo_pulito)  # Output: "Questo è un esempio di testo con una data inclusa."


Questo è un esempio di testo con una data  inclusa.


In [ ]:
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from datetime import datetime

#https://www.medicitalia.it/consulti/?pagina=2&tag=colon-irritabile
#https://www.medicitalia.it/consulti/?pagina={i}&tag=cefalea
#https://www.medicitalia.it/consulti/?pagina={i}&tag=alimentazione
driver = web_driver()
urls = []
data = []
answer_date = None
Location = None
#Una pool di parole per ripulire la stringa della risposta
parole_chiave = ["www.", "http://", "https://", "Dr." ,"Dott.", "Dott.ssa", "Cordiali", "Saluti", "Cordialmente", "Prego", "Arrivederci", "Prof."]

for i in range(1,60):
    URL = f'https://www.medicitalia.it/consulti/?pagina={i}&tag=alimentazione'
    print(URL)
    driver.get(URL)
    wait = WebDriverWait(driver, 40)

    try:
        # Attempt to click the accept all button, with a timeout of 10 seconds
        wait.until(EC.element_to_be_clickable((By.ID,'pt-accept-all'))).click()
    except TimeoutException:
        # If the button is not found within the timeout, print a message and continue.
        print("Accept all button not found, continuing...")

    links = wait.until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR,"a.titconsulto[href]")))
    urls = []
    for link in links:
        urls.append(link.get_attribute('href'))
    for url in urls:
        driver.get(url)
        wait.until(EC.presence_of_element_located((By.XPATH,'//*[@id="question"]')))

        try:
            question = driver.find_element(By.XPATH, '//div[@class="col cons px-4 pb-0"]').text
            question_pulita = rimuovi_data_ora(question)
            # Sostituisci gli a capo con uno spazio
            question_senza_acapo = question_pulita.replace('\n', ' ')
            # Rimuovi eventuali spazi extra testo_senza_acapo = ' '
            question_senza_acapo = ' '.join(question_senza_acapo.split())

        except:
            question = None
            question_pulita = None
            testo_senza_acapo = None

        try:
            answer = driver.find_element(By.XPATH, "//div[contains(@class, 'col cons px-4 pt-4 pb-0')]").text
            stringa_pulita = pulisci_stringa_con_vettore(answer, parole_chiave)
            # Sostituisci gli a capo con uno spazio
            answer_senza_acapo = stringa_pulita.replace('\n', ' ')
            # Rimuovi eventuali spazi extra testo_senza_acapo = ' '
            answer_senza_acapo = ' '.join(answer_senza_acapo.split())

        except:
            answer = None
            stringa_pulita = None
            answer_senza_acapo = None

        try:
            dottore = driver.find_element(By.XPATH, "//span[@class='d-block text-dark-blue nomespec']").text
            if dottore is not None:
              # Rimuovi i titoli "Dott.ssa", "Dott." e "Dr." dal nome
              dottore = dottore.replace('Dott.ssa ', '').replace('Dott. ', '').replace('Dott. ', '').replace('Dr. ', '')
        except:
            dottore = None

        try:
            Specialization = driver.find_element(By.XPATH, "//span[@class='spec-spec d-block text-dark-blue']").text
        except:
            Specialization = None

        try:
            question_date = driver.find_element(By.XPATH, '//time[@class="text-secondary small"]').text
            if question_date is not None:
              # Converti la data in un oggetto datetime
              data_datetime = datetime.strptime(question_date, "%d.%m.%Y %H:%M")
              # Rimuovi l'ora e mantieni solo la data
              data_solo_data = data_datetime.date()
              # Converti la data in un formato ISO 8601 (senza ora) data_iso
              data_iso = data_solo_data.isoformat()
        except:
            question_date = None
            data_iso = None


        print("Question: {}".format(question_senza_acapo))
        print("Question Date: {}".format(data_iso))
        print("Answer: {}".format(answer_senza_acapo))
        print("Doctor: {}".format(dottore))
        print("Specialization: {}".format(Specialization))
        print("Location: {}".format(Location))
        print(i)

        data.append({
            "URL": url,
            "Question Date" : data_iso,
            "Question": question_senza_acapo,
            "Answer Date": answer_date,
            "Answer": answer_senza_acapo,
            "Doctor": dottore,
            "Specialization": Specialization,
            "Location": Location
        })

#driver.find_element(By.CSS_SELECTOR, ".page-numbers > .icon-chevron-right").click()


driver.quit()

# Create DataFrame
df = pd.DataFrame(data)

Output streaming troncato alle ultime 5000 righe.
Location: None
24
Question: Buongiorno, sono una ragazza di 25 anni, uso Nuvaring da 5 anni e periodicamente ho recidive di candida. La cura tra l'altro prevede l'assunzione di integratori, lavande con gine tantum e lavaggi con Pevaryl. Vorrei cortesemente sapere: 1) Pevaryl schiuma all 1% interagisce con Nuvaring ? 2) Le lavande Gine Tantum Rosa allo 0,1% interagiscono con Nuvaring? 3) Gli integratori che uso contengono lactobacillus acidophilus La14, bifidobacterium lactis BIO4, lactobacillus plantarum Lp115, bifidobacterium breve Bb03, magnesio stearato, biossido di titanio, cellulosa microcristallina, gelatina alimentare: questo integratore interagisce con Nuvaring? 4) L'utilizzo simultaneo di questi prodotti interferisce con Nuvaring? Per tutte le domande mi riferisco ovviamente alla riduzione e/o compromissione dell'efficacia contraccettiva di Nuvaring. Grazie anticipatamente, siete sempre molto gentili e professionali.
Question D

In [ ]:
df.head(5)

,URL,Question Date,Question,Answer Date,Answer,Doctor,Specialization,Location
0,https://www.medicitalia.it/consulti/gastroente...,2012-08-18,"Buonasera dottori, avrei un dubbio (che ormai ...",None,"Gentile signore, le preparazioni effervescenti...",Alessandro Scuotto,"Gastroenterologo, Perfezionato in medicine non...",None
1,https://www.medicitalia.it/consulti/medicina-g...,2017-04-08,Sono una donna di 40 anni e soffro da sempre d...,None,"Salve, tutti i sali del Magnesio, non solo il ...",None,None,None
2,https://www.medicitalia.it/consulti/pediatria/...,2009-01-03,"Salve, sto allattando la mia bimba che ha circ...",None,Gentile Signora attraverso l'allattamento mate...,Dr.ssa Agnesina Pozzi,"Pediatra, Perfezionato in medicine non convenz...",None
3,https://www.medicitalia.it/consulti/urologia/3...,2013-05-07,"Salve, da diversi anni soffro di disturbi urin...",None,"Gentile Signore, se dei dubbi si possono avere...",Paolo Piana,Urologo,None
4,https://www.medicitalia.it/consulti/gastroente...,2020-01-29,"Buon pomeriggio dottori, sono una ragazza di 2...",None,A parte il fatto che il ferro deve essere assu...,Marco Bacosi,Gastroenterologo,None


In [ ]:
#df.to_csv("/content/drive/MyDrive/medicinaitalia_dataset_cefalea.csv", index=False)
#df.to_csv("/content/drive/MyDrive/medicinaitalia_dataset_colon.csv", index=False)
df.to_csv("/content/drive/MyDrive/medicinaitalia_dataset_alimentazione.csv", index=False)

# Merge e Pre-Processing dei dati

##MERGE

In [ ]:
import pandas as pd
import os

# Lista per contenere i DataFrame
dataframe = []

# Directory dei file CSV
csv_directory = "/content/drive/MyDrive/"


# Specifica i file CSV con percorsi corretti
csv_files = [
    #Dati dal sito Dica33
    os.path.join(csv_directory, "dica33_dataset_mente.csv"),
    os.path.join(csv_directory, "dica33_dataset_cuore.csv"),
    os.path.join(csv_directory, "dica33_dataset_fegato.csv"),
    os.path.join(csv_directory, "dica33_dataset_mente.csv"),
    os.path.join(csv_directory, "dica33_dataset_naso.csv"),
    #Dati dal sito medicinaitalia
    os.path.join(csv_directory, "medicinaitalia_dataset_alimentazione.csv"),
    os.path.join(csv_directory, "medicinaitalia_dataset_cefalea.csv"),
    os.path.join(csv_directory, "medicinaitalia_dataset_colon.csv"),
    #Dati dal sito miodottore
    os.path.join(csv_directory, "miodottore_dataset_ansia.csv"),
    os.path.join(csv_directory, "miodottore_dataset_depressione.csv"),
]


# Leggi tutti i CSV e aggiungili alla lista
for file in csv_files:
    df = pd.read_csv(file)
    dataframe.append(df)

# Combina tutti i DataFrame in uno solo
merge_df = pd.concat(dataframe, ignore_index=True)

# Salva il file combinato
output_path = os.path.join(csv_directory, "combined_file.csv")
merge_df.to_csv(output_path, index=False)

print(f"Tutti i CSV sono stati combinati e salvati in: {output_path}")


Tutti i CSV sono stati combinati e salvati in: /content/drive/MyDrive/combined_file.csv


In [ ]:
merge_df

,URL,Question Date,Question,Answer Date,Answer,Doctor,Job,Specialization,Location
0,https://www.dica33.it/esperto-risponde/domanda...,2024-10-18,Buongiorno. Mi sono sottoposta a una risonanza...,2011-10-21,NaN,NaN,NaN,NaN,NaN
1,https://www.dica33.it/esperto-risponde/domanda...,2024-10-20,"Buon giorno, a febbraio 2024 ho fatto una angi...",2011-10-21,NaN,NaN,NaN,NaN,NaN
2,https://www.dica33.it/esperto-risponde/domanda...,2024-10-23,"Gentili dottori, a seguito di debolezza alle g...",2011-10-21,NaN,NaN,NaN,NaN,NaN
3,https://www.dica33.it/esperto-risponde/domanda...,2024-08-22,"Buonasera, assumo benzodiazepine da più di 45 ...",2011-10-21,"Gentile utente, la ringrazio per la sua domand...",FEDERICO BARANZINI,NaN,NaN,NaN
4,https://www.dica33.it/esperto-risponde/domanda...,2024-08-23,Buongiorno Dottori attualmente sono in cura pr...,2011-10-21,"Buongiorno, come medico psichiatra, comprendo ...",FEDERICO BARANZINI,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
9493,https://www.miodottore.it/domande-risposte/pre...,NaN,Prendo il brentellix da 8 mesi 5mg al giorno\n...,NaN,"Gentile utente, per qualsiasi richiesta a live...",Ilaria Rasi,NaN,"Psicologo, Psicoterapeuta, Psicologo clinico",Bologna
9494,https://www.miodottore.it/domande-risposte/sal...,NaN,"Salve, sono uno studente 25enne universitario,...",NaN,"Carissimo, per quando riguarda il trattamento ...",Ilaria Rasi,NaN,"Psicologo, Psicoterapeuta, Psicologo clinico",Bologna
9495,https://www.miodottore.it/domande-risposte/sal...,NaN,"Salve, mia madre ha 90 anni e non riesce a dor...",NaN,Immagino che l'insonnia di sua madre sia diffi...,Alessandra D'Antonio,NaN,"Psicologo, Psicologo clinico, Psicoterapeuta",Nocera Inferiore
9496,https://www.miodottore.it/domande-risposte/ho-...,NaN,"Ho iniziato la cura con sertralina DOC, prima ...",NaN,"Buongiorno, tra i sintomi del sovradosaggio di...",Claudia Gervasoni,NaN,"Psicologo, Psicoterapeuta",Roma


In [ ]:
# Salva il file combinato su Google Drive
output_path = "/content/drive/My Drive/dataset_completo.csv"
merge_df.to_csv(output_path, index=False)

print(f"Tutti i CSV sono stati combinati e salvati in: {output_path}")

Tutti i CSV sono stati combinati e salvati in: /content/drive/My Drive/dataset_completo.csv


## Pre-processing finale per i valori nulli sul Question/Answer

In [ ]:
# Verifica i valori nulli nelle colonne "Question" e "Answer"
null_values_question = merge_df['Question'].isnull().sum()
null_values_answer = merge_df['Answer'].isnull().sum()

# Stampa i risultati
print(f"Valori nulli nella colonna 'Question': {null_values_question}")
print(f"Valori nulli nella colonna 'Answer': {null_values_answer}")

Valori nulli nella colonna 'Question': 4
Valori nulli nella colonna 'Answer': 193


In [ ]:
# Rimuovi le righe dove la colonna 'Question' o 'Answer' contiene valori nulli
df_new = merge_df.dropna(subset=['Question', 'Answer'])

In [ ]:
# check ulteriore
null_values_question = df_new['Question'].isnull().sum()
null_values_answer = df_new['Answer'].isnull().sum()

# Stampa i risultati
print(f"Valori nulli nella colonna 'Question': {null_values_question}")
print(f"Valori nulli nella colonna 'Answer': {null_values_answer}")

Valori nulli nella colonna 'Question': 0
Valori nulli nella colonna 'Answer': 0


In [ ]:
df_new

,URL,Question Date,Question,Answer Date,Answer,Doctor,Job,Specialization,Location
3,https://www.dica33.it/esperto-risponde/domanda...,2024-08-22,"Buonasera, assumo benzodiazepine da più di 45 ...",2011-10-21,"Gentile utente, la ringrazio per la sua domand...",FEDERICO BARANZINI,NaN,NaN,NaN
4,https://www.dica33.it/esperto-risponde/domanda...,2024-08-23,Buongiorno Dottori attualmente sono in cura pr...,2011-10-21,"Buongiorno, come medico psichiatra, comprendo ...",FEDERICO BARANZINI,NaN,NaN,NaN
5,https://www.dica33.it/esperto-risponde/domanda...,2024-09-18,Buongiorno sono un uomo di 41 anni da anni ho ...,2011-10-21,"Buongiorno, come psichiatra, comprendo la sua ...",FEDERICO BARANZINI,NaN,NaN,NaN
6,https://www.dica33.it/esperto-risponde/domanda...,2024-09-07,"Buongiorno, mi sono appena iscritta al gruppo!...",2011-10-21,"Gentile Utente, la ringrazio per aver condivis...",FEDERICO BARANZINI,NaN,NaN,NaN
7,https://www.dica33.it/esperto-risponde/domanda...,2024-09-15,Gent. Dott. Ho 64 anni. Ho sempre goduto di bu...,2011-10-21,"Gentile signora, con tutti i limiti di un cons...",PIERGIORGIO BIONDANI,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
9493,https://www.miodottore.it/domande-risposte/pre...,NaN,Prendo il brentellix da 8 mesi 5mg al giorno\n...,NaN,"Gentile utente, per qualsiasi richiesta a live...",Ilaria Rasi,NaN,"Psicologo, Psicoterapeuta, Psicologo clinico",Bologna
9494,https://www.miodottore.it/domande-risposte/sal...,NaN,"Salve, sono uno studente 25enne universitario,...",NaN,"Carissimo, per quando riguarda il trattamento ...",Ilaria Rasi,NaN,"Psicologo, Psicoterapeuta, Psicologo clinico",Bologna
9495,https://www.miodottore.it/domande-risposte/sal...,NaN,"Salve, mia madre ha 90 anni e non riesce a dor...",NaN,Immagino che l'insonnia di sua madre sia diffi...,Alessandra D'Antonio,NaN,"Psicologo, Psicologo clinico, Psicoterapeuta",Nocera Inferiore
9496,https://www.miodottore.it/domande-risposte/ho-...,NaN,"Ho iniziato la cura con sertralina DOC, prima ...",NaN,"Buongiorno, tra i sintomi del sovradosaggio di...",Claudia Gervasoni,NaN,"Psicologo, Psicoterapeuta",Roma


In [ ]:
import re
# Funzione di normalizzazione che rimuove spazi e tutto ciò che inizia con una parentesi tonda
def normalize_city(city):
    # Controlla se la città è una stringa
    if isinstance(city, str):
        # Rimuove gli spazi bianchi e tutto il testo a partire dalla prima parentesi tonda
        city_cleaned = re.sub(r"\s*\([^)]*\)", "", city).strip()

        # Converte tutto in minuscolo e restituisce la città normalizzata
        return city_cleaned.lower()
    return city

# Normalizza la colonna 'Città': rimuove spazi bianchi e ciò che segue una parentesi tonda
df_new['Location'] = df_new['Location'].apply(normalize_city)

# Visualizza il DataFrame dopo la normalizzazione
print("DataFrame dopo la normalizzazione:")
print(df)


DataFrame dopo la normalizzazione:
                                                   URL  Question Date  \
0    https://www.miodottore.it/domande-risposte/ho-...            NaN   
1    https://www.miodottore.it/domande-risposte/buo...            NaN   
2    https://www.miodottore.it/domande-risposte/buo...            NaN   
3    https://www.miodottore.it/domande-risposte/io-...            NaN   
4    https://www.miodottore.it/domande-risposte/sal...            NaN   
..                                                 ...            ...   
295  https://www.miodottore.it/domande-risposte/pre...            NaN   
296  https://www.miodottore.it/domande-risposte/sal...            NaN   
297  https://www.miodottore.it/domande-risposte/sal...            NaN   
298  https://www.miodottore.it/domande-risposte/ho-...            NaN   
299  https://www.miodottore.it/domande-risposte/buo...            NaN   

                                              Question  Answer Date  \
0    Ho 62 anni ,

<ipython-input-20-c41b79444f4c>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['Location'] = df_new['Location'].apply(normalize_city)


In [ ]:
# Salva il DataFrame pulito in un nuovo file CSV
df_new.to_csv('/content/drive/MyDrive/dataset_filtrato.csv', index=False)
